In [3]:
def primary_with_store(primary_key_table,preprocess_store_info_df ):
    """
    Joins the primary_key_table with store_features based on the \"stg_outlet_cd\" column.

    Args:
        primary_key_table : The primary dataframe (primary_key_table).
        preprocess_store_info : The store features dataframe.

    Returns:
        pyspark.sql.DataFrame: The joined dataframe.
    """
    
    joined_df = (
        primary_key_table
        .join(
            preprocess_store_info_df,
            on=["stg_outlet_cd"],
            how="left")
    )
    return joined_df

In [1]:
def joined_df(store_features,sales_features,items_features,time_features):
  joined_df = (
    store_features
    .join(
        sales_features,
        on=["week", "stg_outlet_cd", "stg_item_category_desc_txt"],
        how="left"
         )
     .join(
         items_features,
         on=["week", "stg_outlet_cd", "stg_item_category_desc_txt"],
         how="left"
         )
      .join(
          time_features,
          on=["week", "stg_outlet_cd", "stg_item_category_desc_txt"],
          how="left"
          )
      )
  return joined_df


In [16]:
def create_master_table(joined_df,primary_key_table_with_target_variable):
  master_table=(
      joined_df
      .join(
          primary_key_table_with_target_variable,
          on=["week","stg_outlet_cd","stg_item_category_desc_txt"],
          how="left")
  )
  columns_to_drop= ["weekly_sales_qty","week_date"]
  master_table=master_table.drop(*columns_to_drop)
  master_table = master_table.filter(master_table["next_week_qty"] != 0)
  return master_table

In [27]:
def drop_first_three_weeks(master_table):
    # Find the minimum week date
    min_week_date = master_table.agg(f.min("week")).collect()[0][0]
    
    # Define the date format
    date_format = "%Y-%m-%d"
    
    # Add 21 days to the minimum week date
    new_date_obj = min_week_date + timedelta(days=7*3)
    
    # Convert the new date object to a string
    new_date_string = new_date_obj.strftime(date_format)
    
    # Filter the master table based on the new date
    df_filtered_master_table = master_table.filter(f.col("week") >= new_date_string)
    
    # Return the filtered DataFrame
    return df_filtered_master_table